In [2]:
import os
os.chdir('..')

# 使用metroBJ数据集测试各个模型的序列预测性能
import torch.nn as nn

def main():

    # 1. 读取数据
    INPUT_LENGTH = 24  # 输入序列长度
    OUTPUT_LENGTH = 12  # 输出序列长度
    NUM_STATIONS = 276  # 地铁站点数（特征数）

    from data.BJinflow import BJInflowDataModule

    datamodule = BJInflowDataModule(
        input_length=INPUT_LENGTH,
        output_length=OUTPUT_LENGTH,
        batch_size=1,
        train_val_test_split=(0.7, 0.2, 0.1),
    )

    # 2. 选择骨干模型
    from pretrained import Chronos

    chronos_backbone = Chronos(
        size="tiny",
        task="forecasting",
        out_seq_len=OUTPUT_LENGTH,
        device_map="cpu",
    )

    # 3. 选择任务框架
    from frameworks import ForecastingFramework

    chronos_framework = ForecastingFramework(
        backbone=chronos_backbone,
        custom_head=nn.Identity(),  # 因为要使用backbone的0-shot能力，因此不需要额外的头部
        # 以下参数其实没用：
        backbone_out_features=NUM_STATIONS,
        out_seq_len=OUTPUT_LENGTH,
        out_features=NUM_STATIONS,
    )

    # 4. 训练模型
    chronos_framework.fit(
        datamodule=datamodule, max_epochs=0, lr=1e-3, accelerator='cpu'
    )  # 仅用于初始化Trainer
    chronos_framework.test(datamodule=datamodule)

if __name__ == "__main__":
    main()

/Users/sunluohao/miniforge3/envs/tsfm/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'backbone' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['backbone'])`.
/Users/sunluohao/miniforge3/envs/tsfm/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'custom_head' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['custom_head'])`.


No hyperparameters found in the backbone. Save your hyperparameters in backbone.hparams.

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/sunluohao/miniforge3/envs/tsfm/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


┏━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name                ┃ Type     ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _framework_backbone │ Chronos  │      1 │ train │
│ 1 │ _framework_neck     │ Identity │      0 │ train │
│ 2 │ _framework_head     │ Identity │      0 │ train │
└───┴─────────────────────┴──────────┴────────┴───────┘

Trainable params: 1                                                                                                
Non-trainable params: 0                                                                                            
Total params: 1                                                                                                    
Total estimated model params size (MB): 0

...

/Users/sunluohao/miniforge3/envs/tsfm/lib/python3.11/site-packages/chronos/chronos.py:181: UserWarning: 
torch.searchsorted(): input value tensor is non-contiguous, this will lower the performance due to extra data copy 
when converting non-contiguous tensor to contiguous, please use contiguous input value tensor if possible. This 
message will only appear once per program. (Triggered internally at 
/Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/BucketizationUtils.h:34.)
  torch.bucketize(

/Users/sunluohao/miniforge3/envs/tsfm/lib/python3.11/site-packages/chronos/chronos.py:210: TracerWarning: 
Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of 
Python values, so this value will be treated as a constant in the future. This means that the trace might not 
generalize to other inputs!
  if length > self.config.context_length:

/Users/sunluohao/miniforge3/envs/tsfm/lib/python3.11/site-packages/transformers/generation/utils.py:1479: 
TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if
you use this function to create tensors out of constant variables that would be the same every time you call this 
function. In any other case, this might cause the trace to be incorrect.
  return torch.tensor(token, device=device, dtype=torch.long)

/Users/sunluohao/miniforge3/envs/tsfm/lib/python3.11/site-packages/transformers/generation/utils.py:1513: 
TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the 
data flow of Python values, so this value will be treated as a constant in the future. This means that the trace 
might not generalize to other inputs!
  if eos_token_id is not None and torch.isin(elements=eos_token_id, test_elements=pad_token_id).any():

/Users/sunluohao/miniforge3/envs/tsfm/lib/python3.11/site-packages/transformers/generation/utils.py:1526: 
TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the 
data flow of Python values, so this value will be treated as a constant in the future. This means that the trace 
might not generalize to other inputs!
  if eos_token_id is not None and (torch.is_floating_point(eos_token_id) or (eos_token_id < 0).any()):